In [1]:
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import utils

In [7]:
portfolio, market = utils.load_data()
NOW = utils.set_now()

In [90]:
def cashflow(rate=0, maturity=1, volume=1, payment_type='BULLET'):
    if payment_type == 'BULLET':
        interest = np.full(maturity, rate) * volume
        capital = np.zeros(maturity)
        capital[-1] = volume
        remaining = np.full(maturity, volume)-capital
        cashflow = capital + interest
    elif payment_type == 'LINEAR':
        # Calculate capital
        capital = np.repeat(1/maturity, maturity) * volume
        # Calculate remaining balance after each capital payment
        remaining = np.hstack([volume, volume - np.cumsum(capital)])
        # Calculate interest payment for each period
        interest = remaining[:maturity] * rate
        # Calculate total cash flow for each period
        cashflow = capital + interest
#     elif payment_type == 'ANNUITY':
#         interest = np.zeros(maturity)
#         capital = np.zeros(maturity)
#         cashflow = np.zeros(maturity)
#         remaining = volume
#         if isinstance(rate, (int, float)):
#             rate = np.full(maturity, rate)
#         for i in range(maturity):
#             discount_factor = 1 - (1 + rate[i]) ** -(maturity - i + 1)
#             cashflow[i] = (rate[i] / discount_factor) * remaining if rate[i] != 0 else remaining / (maturity - i + 1)
#             interest[i] = remaining * rate[i]
#             capital[i] = cashflow[i] - interest[i]
#             remaining -= capital[i]

    cf_df = pd.DataFrame({'cashflow': cashflow, 'interest': interest, 'capital': capital, 'remaining': remaining})
    return cf_df

In [91]:
bull = cashflow(rate,maturity,volume,payment_type="BULLET")

In [58]:
volume = 75000
maturity = 36
rate = 0.01

In [102]:
interest = np.zeros(maturity)
capital = np.zeros(maturity)
cashflow = np.zeros(maturity)
remaining = np.zeros(maturity + 1)
remaining[0] = volume

if not isinstance(rate, list):
    rate = np.full(maturity, rate)

# for i in range(maturity):
#     cashflow[i] = (rate[i] / (1 - (1 + rate[i]) ** -(maturity - i + 1))) * remaining[i]
    
#     if rate[i] == 0:
#         cashflow[i] = 1 / (maturity - i + 1) * remaining[i]
    
#     interest[i] = remaining[i] * rate[i]
#     capital[i] = cashflow[i] - interest[i]
#     remaining[i + 1] = remaining[i] - capital[i]

In [11]:
TYPE = 'EUR01'

In [22]:
data = market[market['type']==TYPE].copy()
data['maturity'] = data['date'].apply(lambda date: utils.time_difference_years(date, NOW))

In [14]:
data

,type,date,rate,comment,maturity_years
0,EUR01,2014-09-01,0.300000,1M,-0.079398
1,EUR01,2014-12-01,0.336256,3M,0.169747
2,EUR01,2015-03-01,-2.353646,6M,0.416153
3,EUR01,2015-09-01,-5.691876,1Y,0.919918
4,EUR01,2016-09-01,-5.654177,2Y,1.921971
5,EUR01,2017-09-01,1.015958,3Y,2.921287
6,EUR01,2018-09-01,12.109512,4Y,3.920602
7,EUR01,2019-09-01,25.950572,5Y,4.919918
8,EUR01,2020-09-01,41.283558,6Y,5.921971
9,EUR01,2021-09-01,57.182677,7Y,6.921287


In [ ]:
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve

y = NelsonSiegelSvenssonCurve(0.028, -0.03, -0.04, -0.015, 1.1, 4.0)
t = np.linspace(0, 20, 100)
plt.plot(t, y(t))

In [30]:
from scipy.optimize import curve_fit


def svensson_model(x, beta0, beta1, beta2, tau1, tau2, spot_rate_level):
    # x is already in years (maturities)
    return spot_rate_level + beta0 * (1 - np.exp(-x / tau1)) / (x / tau1) + beta1 * ((1 - np.exp(-x / tau1)) / (x / tau1) - np.exp(-x / tau1)) + beta2 * ((1 - np.exp(-x / tau2)) / (x / tau2) - np.exp(-x / tau2))

def fit_svensson_yield_curve(market_data):
    x_data = market_data['maturity']  # Maturity values (in years)
    y_data = market_data['rate']  # Yields in basis points
    initial_guess = [0.05, 0.05, 0.05, 1.0, 1.0, 0.05]
    params, covariance = curve_fit(svensson_model, x_data, y_data, p0=initial_guess)
    return tuple(params)

In [31]:
print(*fit_svensson_yield_curve(data))

-453.1606447229056 25437.43343257004 30.20354299486503 908.0486458132697 -12.215735436600369 433.9383694159211
